In [46]:
import numpy as np
import heapq

In [47]:
class Sphere():
    def __init__(self, n, pos, vel, rad, mass):
        self.n = n
        self.pos = pos
        self.vel = vel
        self.rad = rad
        self.mass = mass

    def update(self, new_pos, new_vel):
        self.pos = new_pos
        self.vel = new_vel

def initial_state(nr, positions, velocities, rad, mass):
    IS = []
    for n in range(nr):
        IS.append(Sphere(n,positions[n], velocities[n], rad[n], mass[n]))
    return IS

def check_collision(i,j):
    r = i.pos - j.pos
    v = i.vel - j.vel
    rnorm = np.linalg.norm(r)
    rnorm2 = rnorm*rnorm
    rv = np.dot(r,v)
    rv2 = rv*rv
    vnorm2 = np.linalg.norm(v)*np.linalg.norm(v)
    s = abs(i.rad + j.rad)
    s2 = s*s
    if s < rnorm: #condition 1, eq5
        if rv < 0: # condition 2, eq6
            if rnorm2-rv2/vnorm2 < s2: #condition 3, eq7
                dt = (rnorm2 - s2) / (-rv + np.sqrt(rv2 - (rnorm2-s2)*vnorm2 ))
                if dt > 0:
                    return dt
    return None

def create_heap(IS):
    heap_list = []
    for i in IS:
        for jn in range(i.n+1,len(IS)):
            j = IS[jn]
            dt = check_collision(i,j)
            if dt != None:
                heap_list.append((dt,0,i,j))
    heapq.heapify(heap_list)
    return heap_list


In [48]:
IS = initial_state(4, [np.array([0,0]), np.array([0,1]), np.array([1,0]), np.array([1,1])], [np.array([0,1]), np.array([0,0]), np.array([0,1.2]), np.array([0,0])], 0.1*np.ones(4),0.1*np.ones(4))
heap = create_heap(IS)
print(heap)
heapq.heappush(heap, (0.5, 0, 1))
print(heap)
heapq.heappop(heap)
print(heap)

[(0.6666666666666665, 0, <__main__.Sphere object at 0x7f8e728ada90>, <__main__.Sphere object at 0x7f8e728adeb0>), (0.7999999999999998, 0, <__main__.Sphere object at 0x7f8e728ad940>, <__main__.Sphere object at 0x7f8e728adb20>)]
[(0.5, 0, 1), (0.7999999999999998, 0, <__main__.Sphere object at 0x7f8e728ad940>, <__main__.Sphere object at 0x7f8e728adb20>), (0.6666666666666665, 0, <__main__.Sphere object at 0x7f8e728ada90>, <__main__.Sphere object at 0x7f8e728adeb0>)]
[(0.6666666666666665, 0, <__main__.Sphere object at 0x7f8e728ada90>, <__main__.Sphere object at 0x7f8e728adeb0>), (0.7999999999999998, 0, <__main__.Sphere object at 0x7f8e728ad940>, <__main__.Sphere object at 0x7f8e728adb20>)]


In [49]:
def collision(entry):
    # computes new positions and velocities for collison entry in heap
    
    # new positions
    dt = entry[0] - entry[1] # col time - time included in heap
    posi = entry[2].pos + entry[2].vel*dt
    posj = entry[3].pos + entry[3].vel*dt
    
    # new velocities
    reduced_mass = 2 / (1/entry[2].mass + 1/entry[3].mass) # eq. 12
    unit_vec = (entry[2].pos - entry[3].pos) / (entry[2].rad + entry[3].rad) # eq. 13
    mom_change = reduced_mass*np.dot(unit_vec, entry[2].vel - entry[3].vel)*unit_vec # eq. 12
    veli = entry[2].vel - mom_change/entry[2].mass # eq. 10
    velj = entry[3].vel + mom_change/entry[3].mass # eq. 11
        
    return posi, veli, posj, velj

#### below is yet to be incorportated in simulation ####
def wall_collisions(l, i):
    # collisions with lXl square wall centered at (0,0)
    dts = []
    
    # collision with left wall
    rx = i.pos[0] + l/2
    dt = rx/i.vel[0]
    if dt > 0: # check atom travelling in correct direction
        dts.append(dt)
    else:  
        # collision with right wall
        rx = i.pos[0] - l/2
        dt = rx/i.vel[0]
        dts.append(dt)
    
    # collision with top wall
    ry = i.pos[1] + l/2
    dt = ry/i.vel[1]
    if dt > 0: # check atom travelling in correct direction
        dts.append(dt)
    else:
        # collision with bottom wall
        ry = i.pos[1] - l/2
        dt = ry/i.vel[1]
        dts.append(dt)
        
    return dts

In [50]:
#### initialise state and run collisions up to time T ####
#### throws up error due to lack of wall ####

T = 10
IS = initial_state(4, [np.array([0,0]), np.array([0,1]), np.array([1,0]), np.array([1,1])], [np.array([0,1]), np.array([0,0]), np.array([0,1.2]), np.array([0,0])], 0.1*np.ones(4),0.1*np.ones(4))
L = -np.ones(len(IS)) # last collision time for each atom
heap = create_heap(IS)

t = 0
while t < T:
    # possible collision
    entry = heapq.heappop(heap)
    
    # checking if collision is valid event
    if entry[1] < L[entry[2].n]:
        pass
    elif entry[1] < L[entry[3].n]:
        pass
    else: # collision valid
        
        # updating last collision times
        L[entry[2].n] = entry[0]
        L[entry[3].n] = entry[0]
        
        # new particle pos and vel
        posi, veli, posj, velj = collision(entry)
        
        # update pos and vel
        entry[2].update(posi, veli)
        entry[3].update(posj, velj) # this will change all heap pos and vels
        # may not matter as they're no longer be valid?
        
        # update heap
        for i in IS:
            # collisions with first sphere
            dt = check_collision(i, entry[2])
            if dt != None:
                heapq.heappush(heap, (dt + entry[0],entry[0],i, entry[2]))
            # collisions with second
            dt = check_collision(i, entry[3])
            if dt != None:
                heapq.heappush(heap, (dt + entry[0],entry[0],i, entry[3]))
                
        # update time counter
        t = entry[0]

IndexError: index out of range